## One- and two-sample t-tests

Although this may seem too rudimentary, it is a great place to start learning about different model parameterizations and setting up contrasts.

Goals
*  Learn how to set up 1- and 2-sample t-tests using OLS regression
*  Understand how to set up a simple contrast
*  Learn how to parameterize the 2-sample t-test in different ways when using regression

The equivalent parameterization important to understand for a couple of reasons.  First, it is nice to have options when one may feel more intuitive to use than other.  Second, because it is often misunderstood that a reparametrization of a model is doing something "better".  A common example is to set up regressors differently to avoid collinearity between regressors, as this usually will not change a thing about your inferences (it *shouldn't* in most cases).  Also, we'll see later on that the collinearity between regressors can be less important than the collinearity of the contrast of interest with the rest of the model regressors and this will be covered later.!

Relevant YouTube videos (if you need more details):
* [Why does a design matrix with a column of 1s estimate the group mean?](https://www.youtube.com/watch?v=_zWFMNCzdFQ)  This was a question  I was asked during a summer course and I figure if one person wondered why it was the mean (not the median or something else) other people might wonder too.  I do *not* cover this below.
* [Two-sample t-test](https://www.youtube.com/watch?v=TshYzZWNnHA)


#### One-sample t-test

The intercept-only model is the model one uses for running a 1-sample t-test.  There's really no use for a contrast in this example, but one is run as an introduction to the syntax used in sm.OLS.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
np.random.seed(1896) #Year William Sealy Gosset was born (pseudonym: Student. 
# English statistician, chemist and brewer who served as Head Brewer of Guinness
# and Head Experimental Brewer of Guinness and was a pioneer of modern statistics)
nsub = 40
dat_1sampt = np.random.normal(size=nsub)

In [3]:
x_1sampt = pd.DataFrame({'intercept': np.ones(nsub)})
mod_1sampt = sm.OLS(dat_1sampt, x_1sampt).fit()
mod_1sampt.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
No. Observations:                  40   F-statistic:                       nan
Covariance Type:            nonrobust   Prob (F-statistic):                nan
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.3548      0.160      2.220      0.032       0.032       0.678
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

A contrast is overkill in this situation, but here is what the syntax looks like.  To construct a contrast first think of what you're trying to test in terms of the alternative.  The constrast-based tests using sm.OLS are all 2-sided tests (alternative is either greater than or less than 0), so no need to focus on directionality.  Note that fMRI analyses are almost always 1-sided, but this will be discussed later on.

In this case the test is to see if  $\beta_0 \neq 0$.  The next step is to set up your parameter vector, which in this case is a single parameter [$\beta_0$] and the contrast is a vector such that the dot product of the parameter vector with the contrast vector yields your alternative hypothesis expression.  In this case it would be [1] since [1]$\cdot$[$\beta_0$] = $\beta_0$.  

The contrast is then entered into the ``t_test`` attribute as shown here.  Obviously, in this case, this is redundant since this information was in the model summary.

In [4]:
mod_1sampt.t_test([1])

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.3548      0.160      2.220      0.032       0.032       0.678

### Two-sample t-tests (3 ways)

All of these models are equivalent.  What it meant by that is the model fits will be identical and although the parameter estimates may differ and have different interpretations, but contrasts of the parameter estimates can be used to generate equivalent tests.  Equivalent models will always have the same number of regressors (necessary but not sufficient for equivalence), so all of the models here will have 2 regressors. 

#### Rank deficiency
Before we dig into some regression models, it is helpful to know what is meant when a design is said to be "rank deficient".  If you're a Linear Algebra nerd, this simply means the design matrix (matrix made by putting each regressor in a column) is not full column rank.  If you're not a Linear Algebra nerd, this simply means you cannot recreate any single regressor in a design matrix by taking linear combinations of other columns in the design matrix.  For example, this matrix is not full rank since Regressor 2 + Regressor 3 = Regressor 1 (or, Regressor 1 + (-1)*Regressor 2 = Regressor 3, etc.):
| Regressor 1 | Regressor 2 | Regressor 3 |
|:--------:|:--------:|:--------:|
|  1   |  1   |  0   |
|  1   |  1   |  0   |
|  1   |  0   |  1   |
|  1   |  0   |  1   |

Technically matrices that do not have full column rank in regression are a problem because a matrix inversion step is involved and that matrix will only be invertible if the design matrix is full column rank.  That said, some software packages, like SPM, will allow design matrices that are *not* full column rank as they use special matrix tricks for estimation, but they will fail if the *contrast* is rank deficient.  It is really hard to know if a contrast is or isn't rank deficient, so this approach can be confusing and, in my opinion, can make it harder to understand what the interpretations of individual parameters in a model are, which makes contrast construction difficult.  Whether or not a contrast is rank deficient is an advanced topic and we'll talk about this idea more when discussing the Variance Inflation Factor of contrasts later on.

Let's make some data!


In [5]:
np.random.seed(1912) #Alan Turing was born in 1912.  If you're not famliar with his story, go read about it because it is important.
nsub_per_group = 40
groups = np.repeat([0,1], nsub_per_group)
dat_2sampt = 3 + .25 * groups + np.random.normal(size=2*nsub_per_group)

#### Model A: Intercept and 1 group indicator
This style of model is common when using software that automatically includes an intercept in the model; for example, lm() in R.  For the purposes of understanding what the parameters of the model mean, it is helpful to write out the regression.  In this case the model is
$$Y = \beta_0 + \beta_1G_1 + \epsilon,$$
where $G_1$ is an indicator variable that has a value of 0 when a subject is in Group 0 and a value of 1 when a subject is in Group 1 and $\epsilon$ is random error.  To understand the parameter interpretations when the regressors are *not* continuous regressors (e.g. age, IQ, RT) just plug in values for the regressor. Here we'll plug in the possibilities for the $G_1$ regressor and see what we get. 

G_1=0: $Y=\beta_0$, so this is the estimated value of $Y$ for a subject in group 0.  Was that a surprise?

G_1=1: $Y=\beta_0 + \beta_1$, so this is the estimated value of $Y$ for a subject in group 1.

$\beta_1$: Putting the above two results together, $\beta_1$ is the mean of G1 - mean of G2.  If the test is statistically significant, the sign of $\beta_1$ indicates whether G1 > G2 (positive) or G1 < G2 (negative).

In [6]:
x_2sampt_a = pd.DataFrame({'intercept': np.ones(2*nsub_per_group),
                           'group0_1': groups})


In [7]:
mod_2sampt_a = sm.OLS(dat_2sampt, x_2sampt_a).fit()
mod_2sampt_a.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.012
No. Observations:                  80   F-statistic:                   0.04286
Covariance Type:            nonrobust   Prob (F-statistic):              0.837
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      3.2000      0.165     19.394      0.000       2.872       3.529
group0_1      -0.0483      0.233     -0.207      0.837      -0.513       0.416
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

A nice thing about this parameterization is a contrast isn't needed to obtain the inference for the 2-sample t-test since the parameter for ``group0_1`` is the group difference, as outlined above.  Below the contrasts for the 2-sample t-test, Group 0 mean and Group 1 mean are given.

In [8]:
print('2-sample t-test result')
print(mod_2sampt_a.t_test([0, 1]))

mod_2sampt_a_effect = np.round(mod_2sampt_a.t_test([0, 1]).effect[0], 3)
mod_2sampt_a_sd = np.round(mod_2sampt_a.t_test([0, 1]).sd[0, 0], 3)
mod_2sampt_a_p = np.round(mod_2sampt_a.t_test([0, 1]).pvalue, 3)

print('\n')
print('Test whether the mean of group 0 is significantly different from 0')
print(mod_2sampt_a.t_test([1, 0]))

print('\n')
print('Test whether the mean of group 1 is significantly different from 0')
print(mod_2sampt_a.t_test([1, 1]))

2-sample t-test result
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0483      0.233     -0.207      0.837      -0.513       0.416


Test whether the mean of group 0 is significantly different from 0
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             3.2000      0.165     19.394      0.000       2.872       3.529


Test whether the mean of group 1 is significantly different from 0
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------

#### Model B: Intercept and 1/-1 regressor

In this example the intercept is included and a 1/-1 variable ($X_{1/-1}$) is used (-1 for group 0 and 1 for group 1).  The 1/-1 style of regressor is common when trying to replicate an ANOVA style analysis using linear regression.  Repeating the earlier process, the model is
$$Y = \beta_0 + \beta_1 X_{1/-1}$$
 
$X_{1/-1} = -1$, subject in Group 0: $Y = \beta_0 - \beta_1$ 

$X_{1/-1} = 1$, subject in Group 1: $Y = \beta_0 + \beta_1$ 

Group 1 - Group 0 = $(\beta_0 + \beta_1) - (\beta_0 - \beta_1) = 2\beta_1$
so the parameter, $\beta_1$, is half the group difference.  Inferences are not impacted by scaling, since the standard error will also be scaled, which willl will see below. Specifically, the inferences for $\beta_1$ in the two models will match, but the parameter estimate and standard error will be half as large in this model.

In the following model output, see that $\beta_1$ matches the previous model, but $\beta_0$ has a different value.  Why?  Any guesses for what the intercept is in this model?

In [9]:
x_pos_neg_1 = np.zeros(groups.shape)
x_pos_neg_1[groups == 0] = -1
x_pos_neg_1[groups == 1] = 1

x_2sampt_b = pd.DataFrame({'intercept': np.ones(2*nsub_per_group),
                           'x_pos_neg_1': x_pos_neg_1})

mod_2sampt_b = sm.OLS(dat_2sampt, x_2sampt_b).fit()
mod_2sampt_b.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.012
No. Observations:                  80   F-statistic:                   0.04286
Covariance Type:            nonrobust   Prob (F-statistic):              0.837
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
intercept       3.1759      0.117     27.220      0.000       2.944       3.408
x_pos_neg_1    -0.0242      0.117     -0.207      0.837      -0.256       0.208
===============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
print('Mean of Group 0')
print(mod_2sampt_b.t_test([1, -1]))

print('\n')
print('Mean of Group 1')
print(mod_2sampt_b.t_test([1, 1]))

print('\n')
print(f'Two-sample t-test \n'
      f'(note Model A found coef = {mod_2sampt_a_effect}, se = {mod_2sampt_a_sd}, p = {mod_2sampt_a_p})')
print(mod_2sampt_b.t_test([0, 1]))

print('\n')
print(f'Two-sample t-test [0, 2] \n'
      f'(note Model A found coef = {mod_2sampt_a_effect}, se = {mod_2sampt_a_sd}, p = {mod_2sampt_a_p})')
print(mod_2sampt_b.t_test([0, 2]))

Mean of Group 0
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             3.2000      0.165     19.394      0.000       2.872       3.529


Mean of Group 1
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             3.1517      0.165     19.101      0.000       2.823       3.480


Two-sample t-test 
(note Model A found coef = -0.048, se = 0.233, p = 0.837)
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0

#### Model C: No intercept and two group indicators

This is probably a "fan favorite" in the fMRI world, since the contrast is the most intuitive and this formulation easily extends to multiple groups of subjects.  The model omits the intercept and uses a group indicator for each group.  Note, if an intercept is added the model will not be full rank and is not estimable.  Generally, when constructing one model that is equivalent to another, you at least need the same number of regressors.

$$Y=\beta_0 Group_0 + \beta_1 Group_1$$

$Y=\beta_0$ for a subject in Group 0

$Y=\beta_1$ for a subject in Group 1

It follows that the two-sample t-test would be given by $\beta_1 - \beta_0$.  

In [11]:
group_0 = np.zeros(groups.shape)
group_0[groups==0] = 1
group_1 = groups

x_2sampt_c = pd.DataFrame({'group0': group_0,
                           'group1': group_1})

mod_2sampt_c = sm.OLS(dat_2sampt, x_2sampt_c).fit()
mod_2sampt_c.summary(slim=True)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.012
No. Observations:                  80   F-statistic:                   0.04286
Covariance Type:            nonrobust   Prob (F-statistic):              0.837
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
group0         3.2000      0.165     19.394      0.000       2.872       3.529
group1         3.1517      0.165     19.101      0.000       2.823       3.480
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
print('Mean of Group 0')
print(mod_2sampt_c.t_test([1, 0]))

print('\n')
print('Mean of Group 1')
print(mod_2sampt_c.t_test([0, 1]))

print('\n')
print('Two-sample t-test')
print(mod_2sampt_c.t_test([-1, 1]))

Mean of Group 0
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             3.2000      0.165     19.394      0.000       2.872       3.529


Mean of Group 1
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             3.1517      0.165     19.101      0.000       2.823       3.480


Two-sample t-test
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0483      0.233     -0.207      0.837      -0.513       0.41

#### Bonus model if you'd like to figure this out

Another way to model this is to mean center the groups variable and use that.  In this case the intercept is the overall mean (nice) and the centered group variable is still the two-sample t-test (also nice).  Can you figure out what contrast would give the mean of G1 and mean of G2?

Note, I wouldn't recommend using this model if you really wanted to pull out the separate group mean estimates (not just p-values or t-stats).  There's too much room for error when the groups are not balanced.  If you're only looking at p-values (probably only "okay" for fMRI data), then it won't matter.

In [13]:
groups_centered = groups - np.mean(groups)
print(groups_centered)

x_2sampt_d = pd.DataFrame({'intercept': np.ones(2*nsub_per_group),
                           'group0_1': groups_centered})

mod_2sampt_d = sm.OLS(dat_2sampt, x_2sampt_d).fit()
mod_2sampt_d.summary(slim=True)

[-0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5
 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5
 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5]


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.012
No. Observations:                  80   F-statistic:                   0.04286
Covariance Type:            nonrobust   Prob (F-statistic):              0.837
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      3.1759      0.117     27.220      0.000       2.944       3.408
group0_1      -0.0483      0.233     -0.207      0.837      -0.513       0.416
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""